# Fermionic Dimer

In [ ]:
using KadanoffBaym
using LinearAlgebra
using LsqFit
using JLD

using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## Model

### Non-Hermitian Hamiltonian:

$$
\begin{align}\begin{split}
    \hat{H} &= \varepsilon_1 c^{\dagger}_1 c^\phantom{\dagger}_1  + \varepsilon_2 c^{\dagger}_2 c^\phantom{\dagger}_2 + J \left(c^{\dagger}_1 c^\phantom{\dagger}_2 + c^{\dagger}_2 c^\phantom{\dagger}_1\right)
\end{split}\end{align}
$$


### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= i \partial_T
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right]    \\
    0 &= i \partial_T
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
\end{split}\end{align}

## Solving

In [ ]:
# final time
T = 5.0

# Hamiltonian
ε₁ = 1.0
ε₂ = -1.0
J = 20.0
h = ComplexF64[ε₁ J; J ε₂];

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the end!
gfL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Lesser)
gfG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Greater)

# Note to Tim: Accessing a gf with 2 indices gives you the gf at those time points
# Lesser[1,1] <=> Lesser[:,:,1,1]

# initial condition
N_0 = 0.1

gfL[1,1,1,1] = 1.0im * N_0
# gfL[1,2,1,1] = 1.0im * N_0
# gfL[2,1,1,1] = 1.0im * N_0
gfG[:, :, 1, 1] = -1.0im .* diagm([1.0, 1.0]) .+ gfL[:, :, 1, 1]
# gfG[1,1,1,1] = -1.0im * (1 - N_0)
# gfG[2,2,1,1] = -1.0im

u0 = [gfL, gfG];

function rhs_vert(u, times, t, t′)
    u1, u2 = u[1], u[2]
    rhs1 = -1.0im * h * u1[t,t′]
    rhs2 = -1.0im * h * u2[t,t′]
  return [rhs1, rhs2]
end

function rhs_diag(u, times, t)
    u1, u2 = u[1], u[2]
    rhs1 = -1.0im * (h * u1[t,t] - u1[t,t] * adjoint(h))
    rhs2 = -1.0im * (adjoint(h) * u2[t,t] - u2[t,t] * h)
  return [rhs1, rhs2]
end

In [ ]:
gfL[:, :, 1, 1]

In [ ]:
# Basically this will mutate Lesser and Greater in place
sol, _ = kbsolve(rhs_vert, rhs_diag, u0, (0.0, T); dtini=1e-8, atol=1e-9, rtol=1e-10);

In [ ]:
# stop = Int(n/2) + 1 

times = sol.t #range(0, length=n + 1, stop=T) |> collect;

n = length(sol.t) - 1

horizontal_times = k -> vcat(times[k:end] .- times[k], times[end] .+ (1:(k - 1) |> collect) .* (times[end] - times[end-1]));

In [ ]:
n

## QuTiP benchmark

### Jordan-Wigner transformation

$$
\begin{align*}
    c^\phantom{\dagger}_1 &\to \sigma^-_1,\\
    c^{\dagger}_1 &\to \sigma^+_1.\\
\end{align*}
$$

$$
\begin{align*}
    c^\phantom{\dagger}_2 &\to (-\sigma^z_1)\otimes\sigma^-_2,\\
    c^{\dagger}_2 &\to (-\sigma^z_1)\otimes\sigma^+_2.\\
\end{align*}
$$

### Simulation

In [ ]:
U = 0*10.0 

In [ ]:
# initial state
psi0_list = [(1/sqrt(2.0)) * (qt.basis(2, 1) + qt.basis(2, 0)), qt.basis(2, 0)]
psi0 = qt.tensor(psi0_list)

# define annihilation operators
c_1_list = [qt.destroy(2), qt.qeye(2)]
c_1  = qt.tensor(c_1_list)    
c_2_list = [qt.destroy(2) * qt.destroy(2).dag() - qt.destroy(2).dag() * qt.destroy(2), qt.destroy(2)]
# c_2_list = [qt.qeye(2), qt.destroy(2)]
c_2  = qt.tensor(c_2_list)    

# Hamiltonian
H  = J * (c_1.dag() * c_2 + c_2.dag() * c_1)
H += ε₁ * c_1.dag() * c_1 + ε₂ * c_2.dag() * c_2; # do not give qutip a non-Hermitian Hamiltonian
H += U * (c_1.dag() * c_1 * c_1.dag() * c_1 + c_2.dag() * c_2 * c_2.dag() * c_2)

observables = [c_1.dag()*c_1, c_2.dag()*c_2];

In [ ]:
c_2.dag() * c_1

In [ ]:
c_1 * c_2.dag()

In [ ]:
c_1.dag() * c_1

In [ ]:
c_1 * c_1.dag()

In [ ]:
c_1 * c_2.dag() == - c_2.dag() * c_1

In [ ]:
# quickly solve once for observables
me = qt.mesolve(H, psi0, times, [], observables)

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, psi0, times); # t_sols.states returns state vectors!

#### Two times

Calculate $\langle c^\dagger_1(t') c^\phantom{\dagger}_1(t)\rangle$ and $\langle c^\phantom{\dagger}_1(t')c^\dagger_1(t) \rangle$, where $t' = t + \tau$.

In [ ]:
# tau_t_sols = Dict()
# for k in 1:length(t_sols.states)
#     tau_t_sols[k] = qt.mesolve(H, c_1 * t_sols.states[k] * t_sols.states[k].dag(), horizontal_times(k)).states
# end

# c_1_dag_c_1 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
# for k in 1:length(t_sols.states)
#     for l in 1:length(t_sols.states)
#         c_1_dag_c_1[k, l] = (c_1.dag() * tau_t_sols[k][l]).tr()    
#     end
# end

# # tau_t_sols = Dict()
# # for k in 1:length(t_sols.states)
# #     tau_t_sols[k] = qt.mesolve(H, c_1.dag() * t_sols.states[k] * t_sols.states[k].dag(), horizontal_times(k)).states
# # end

# # c_1_c_1_dag = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
# # for k in 1:length(t_sols.states)
# #     for l in 1:length(t_sols.states)
# #         c_1_c_1_dag[k, l] = (c_1 * tau_t_sols[k][l]).tr()    
# #     end
# # end

In [ ]:
# tau_t_sols = Dict()
# for k in 1:length(t_sols.states)
#     tau_t_sols[k] = qt.mesolve(H, c_2 * t_sols.states[k] * t_sols.states[k].dag(), times).states
# end

# c_2_dag_c_2 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
# for k in 1:length(t_sols.states)
#     for l in 1:length(t_sols.states)
#         c_2_dag_c_2[k, l] = (c_2.dag() * tau_t_sols[k][l]).tr()    
#     end
# end

# tau_t_sols = Dict()
# for k in 1:length(t_sols.states)
#     tau_t_sols[k] = qt.mesolve(H, c_2.dag() * t_sols.states[k] * t_sols.states[k].dag(), times).states
# end

# c_2_c_2_dag = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
# for k in 1:length(t_sols.states)
#     for l in 1:length(t_sols.states)
#         c_2_c_2_dag[k, l] = (c_2 * tau_t_sols[k][l]).tr()    
#     end
# end

In [ ]:
# # reshape the above array to fit into our two-time "matrix" structure 
# # see the plot below for illustration
# unskewed_c_1_dag_c_1 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
# for (k, x) in enumerate([c_1_dag_c_1[k, :] for k in 1:length(t_sols.states)])
#     for (l, y) in enumerate(x)
#         ind = k + l - 1 # verify the -1 relative to the original python code
#         unskewed_c_1_dag_c_1[k, ind] = y  
#     end
# end

# # unskewed_c_1_c_1_dag = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
# # for (k, x) in enumerate([c_1_c_1_dag[k, :] for k in 1:length(t_sols.states)])
# #     for (l, y) in enumerate(x)
# #         ind = k + l - 1 # verify the -1 relative to the original python code
# #         unskewed_c_1_c_1_dag[k, ind] = y  
# #     end
# # end

In [ ]:
# # reshape the above array to fit into our two-time "matrix" structure 
# # see the plot below for illustration
# unskewed_c_2_dag_c_2 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
# for (k, x) in enumerate([c_2_dag_c_2[k, :] for k in 1:length(t_sols.states)])
#     for (l, y) in enumerate(x)
#         ind = k + l - 1 # verify the -1 relative to the original python code
#         unskewed_c_2_dag_c_2[k, ind] = y  
#     end
# end

# unskewed_c_2_c_2_dag = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
# for (k, x) in enumerate([c_2_c_2_dag[k, :] for k in 1:length(t_sols.states)])
#     for (l, y) in enumerate(x)
#         ind = k + l - 1 # verify the -1 relative to the original python code
#         unskewed_c_2_c_2_dag[k, ind] = y  
#     end
# end

In [ ]:
# figure(figsize=(6, 2))
# subplot(121)
# imshow(real(c_1_dag_c_1), cmap="plasma")

# subplot(122)
# imshow(real(unskewed_c_1_dag_c_1), cmap="plasma")

# tight_layout()

## Plotting

In [ ]:
idx1 = 1
idx2 = 1;

In [ ]:
# Analytic result
f_2(T, tp) = exp(-1.0im * h * horizontal_times(T)[tp]) * exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * sol.t[T])
f(T, tp) = exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * horizontal_times(T)[tp]) * exp(1.0im * h * sol.t[T])
f_diag(T) = exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * sol.t[T])

In [ ]:
figure(figsize=(12, 6))
subplot(221)
plot(sol.t, [imag(gfL.data[idx1, idx2, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="r")
# plot(sol.t, [imag(gfL.data[2, 2, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="r")
# plot(sol.t, me.expect[1], c="k", ls="--", lw=3.0, alpha=0.5)
# plot(sol.t, me.expect[2], c="k", ls="--", lw=3.0, alpha=0.5)
# plot(sol.t, [real(unskewed_c_1_dag_c_1[k, k]) for k in 1:n+1], c="k", ls="--", lw=3.0, alpha=0.5)
plot(sol.t, [imag(f_diag(k)[idx1, idx2]) for k in 1:n+1], c="C0", ls="--", lw=3.0, alpha=0.5)
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(222)
idx = 1
# plot(sol.t[idx:end - 1], [real(unskewed_c_1_dag_c_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", c="k", lw=3.0, alpha=0.5, ms=3)
plot(sol.t[idx:end - 1], [imag(gfL.data[idx1, idx2, idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", ms=1, c="r")
plot(sol.t[idx:end - 1], [imag(f(idx, k))[idx1, idx2] for k in 1:length(sol.t) - idx], "x", c="C0", lw=5.0, alpha=0.5, ms=5)

tight_layout()
# savefig("test.pdf")

In [ ]:
figure(figsize=(12, 6))
subplot(221)
plot(sol.t, [abs.(gfL.data[idx1, idx2, k, k]) for k in 1:length(sol.t)] 
    .- [abs.(f_diag(k)[idx1, idx2]) for k in 1:n+1], marker="", ms=3.0, ls="-", c="r")
# plot(sol.t, me.expect[1] .- [imag(f_diag(k)[1, 1]) for k in 1:n+1], c="k", ls="--", lw=3.0, alpha=0.5)
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(222)
idx = 30
plot(sol.t[idx:end - 1], [abs.(gfL.data[idx1, idx2, idx, k + idx - 1]) for k in 1:length(sol.t) - idx]
    .- [abs.(f(idx, k))[idx1, idx2] for k in 1:length(sol.t) - idx], "o", ms=1, c="r")
# semilogy(sol.t[idx:end - 1], [real(unskewed_c_1_dag_c_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx] .- [imag(f(idx, k))[1, 1] for k in 1:length(sol.t) - idx], "o", c="k", lw=3.0, alpha=0.5, ms=1)

tight_layout()
# savefig("test.pdf")

## Error scaling

In [ ]:
epsilons = [2.0^(-k) for k in 10:42]
init_dt = 1e-16; # epsilons[end]
err_data = [(0.0, 0.0, 0.0) for _ in 1:length(epsilons)]
for (k, eps) in enumerate(epsilons)
    print(k, ", ")
    
    sol, _ = kbsolve(rhs_vert, rhs_diag, u0, (0.0, T); dtini=init_dt, atol=eps, rtol=eps);
    n = length(sol.t) - 1;
    
    # Analytic result
    horizontal_times = x -> vcat(sol.t[x:end] .- sol.t[x], sol.t[end] .+ (1:(x - 1) |> collect) .* (sol.t[end] - sol.t[end-1]));
    f(T, tp) = exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * horizontal_times(T)[tp]) * exp(1.0im * h * sol.t[T])
    f_diag(T) = exp(-1.0im * h * sol.t[T]) * gfL[:, :, 1, 1] * exp(1.0im * h * sol.t[T])
    
    horizontal_err = 0.0
    for idx in 1:n
        horizontal_err += norm([(gfL.data[1, 1, idx, k + idx]) for k in 1:length(sol.t) - idx] 
            .- [(f(idx, k))[1, 1] for k in 1:length(sol.t) - idx + 1][2:end]) # not counting the diagonal
    end    

    err = norm([(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)] .- [(f_diag(k)[1, 1]) for k in 1:n+1])
    err += 2 * horizontal_err
    err_data[k] = (n, (1.0/(n)^2) * err, eps)    
end

save("error_data.jld", "params", [T, h, N_0], "err_data", err_data)

In [ ]:
err_data = load("error_data.jld")["err_data"]
err_data = err_data[3:end];

In [ ]:
xdata = log10.([x[1] for x in err_data])
ydata = log10.([x[2] for x in err_data]);

In [ ]:
func_power = (n, p) -> 10^(p[2]) * n^(-p[1]);
fit_func = (n, p) -> -p[1] .* n .+ p[2];
fit_result = curve_fit(fit_func, xdata, ydata, [2.0, 1]);
coef(fit_result)

In [ ]:
floor(1.003, sigdigits=4)

In [ ]:
figure(figsize=(8, 5))

# func_1 = x -> 8e6x^-6
# func_2 = x -> 2e23x^-13
# func_3 = x -> 6e22x^-16

ax = subplot(221)
plot(xdata, ydata, "s", lw=0, ms=3)
plot(xdata, map(x -> fit_func(x, coef(fit_result)), xdata), "-", lw=2, 
    label="\$\\mathcal{O}(h^{"*string(coef(fit_result)[1] |> x -> floor(x, sigdigits=3))*"})\$")
# xlim(25, 75)
# ylim(-17, 0)
# ax.set_yticks([0, -5, -10, -15])
xlabel("\$\\log(n)\$")
ylabel("\$\\log(\\varepsilon_{\\mathrm{abs}})\$")
legend(frameon=false)

ax = subplot(222)
plot(([x[1] for x in err_data]), ([x[2] for x in err_data]), "s", lw=0, ms=3)
plot(([x[1] for x in err_data]), (map(x -> func_power(x, coef(fit_result)), [x[1] for x in err_data])), "-", lw=2, alpha=0.8)
# xlim(25, 75)
# ylim(-17, 0)
# ax.set_yticks([0, -5, -10, -15])
xlabel("\$n\$")
ylabel("\$\\varepsilon_{\\mathrm{abs}}\$")

tight_layout()

In [ ]:
# figure(figsize=(8, 5))

ax = subplot(221)
semilogx(([x[1] for x in err_data]), log10.([x[2] for x in err_data]), "s", lw=0, ms=3)
# axhline(log10(init_dt), c="grey", ls="--", label="\$\\Delta t_{\\mathrm{init}}\$")
# xlim(1e1, 1e3)
# ylim(-17, 0)
# ax.set_yticks([0, -5, -10, -15])
xlabel("\$n\$")
ylabel("\$\\log(\\varepsilon_{\\mathrm{abs}})\$")
# legend()
tight_layout()

In [ ]:
figure(figsize=(8, 5))

ax = subplot(221)
plot(log10.([x[3] for x in err_data]), log10.([x[1] for x in err_data]), "s")
xlim(-3, -13)
# ylim(-17, 0)
# ax.set_yticks([0, -5, -10, -15])
# ax.set_yticklabels([])
xlabel("\$\\log(\\mathrm{tol})\$")
ylabel("\$\\log(n)\$")

tight_layout()

## Testing